In [1]:
import pandas as pd,seaborn as sns,matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import json
import requests
from bs4 import BeautifulSoup
from modules.functions import get_schedule

In [2]:
get_schedule("Purdue")

,DATE,OPPONENT,GAME_ID
2,"Tue, Nov 8",Milwaukee,401483405
3,"Fri, Nov 11",Austin Peay,401483406
4,"Tue, Nov 15",Marquette,401479696
5,"Thu, Nov 24",West Virginia,401483331
6,"Fri, Nov 25",Gonzaga,401486748
7,"Sun, Nov 27",Duke,401486751
9,"Wed, Nov 30",@ Florida State,401479679
10,"Sun, Dec 4",Minnesota,401484836
11,"Wed, Dec 7",Hofstra,401483407
12,"Sat, Dec 10",@ Nebraska,401484843


In [3]:
game_id = (401483406)
url = 'https://www.espn.com/mens-college-basketball/boxscore/_/gameId/'+str(game_id)
r = requests.get(url)
print(f"Status Code: {r.status_code}")
soup = BeautifulSoup(r.content,'lxml')

Status Code: 200


In [4]:
#Isolate the home team, away team, and game date.  Away team is always first
away_team,home_team = [team.strip().title() for team in [i for i in str(soup.find('title')).split('-') if " vs. " in i][0].replace('helmet="true">',"").split(" vs.")]
game_date = str(soup.find("title")).split("-")[-1].split("|")[0].strip()
print(f"Away Team: {away_team}")
print(f"Home Team: {home_team}")
print(f"Game Date: {game_date}")

Away Team: Austin Peay
Home Team: Purdue
Game Date: November 11, 2022


In [5]:
#Infer tables with Pandas
dfs = pd.read_html(url)
#Away team is always index value 1, home team is 2
away_team_df = dfs[1]
home_team_df = dfs[2]
# Convert columns from tuples to strings
if type(away_team_df.columns[0]) is tuple:
  away_team_df.columns = [i[0] for i in away_team_df.columns.tolist()]
if type(home_team_df.columns[0]) is tuple:
  home_team_df.columns = [i[0] for i in home_team_df.columns.tolist()]

In [25]:
# Pandas pulls in a lot of dataframes
# Filterint to only retrieve the entries we're interested in
away_players,away_stats,home_players,home_stats = dfs[1:5]
# Renaming the column in the home and away players dataframe to "player"
away_players.columns,home_players.columns = ['Player'],['Player']

# Remove entries we don't need
away_players = away_players.iloc[1:len(away_players)-1,]
away_players = away_players.loc[away_players.Player != "bench"]

# Remove entries we don't need
home_players = home_players.iloc[1:len(home_players)-1,]
home_players = home_players.loc[home_players.Player != "bench"]
# Grabbing the last letter from the player column and isolating it into it's own column
# This becomes the position (G,F,C)
home_players['Position'] = [i[-1] for i in home_players.Player]
away_players['Position'] = [i[-1] for i in away_players.Player]
home_players['Player'] = [i[:-2].strip() for i in home_players.Player]
away_players['Player'] = [i[:-2].strip() for i in away_players.Player]

# Pandas doesn't recognize the first row as a header, so I'm manually assigning it to the stats dataframes
away_stats.columns = away_stats.iloc[0,:].tolist()
home_stats.columns = home_stats.iloc[0,:].tolist()

# Removing column break headers
home_stats = home_stats.loc[home_stats.MIN != "MIN"]
away_stats = away_stats.loc[away_stats.MIN != "MIN"]

# Removing the last row as it's all null values
home_stats = home_stats.iloc[:len(home_stats)-1,]
away_stats = away_stats.iloc[:len(away_stats)-1,]

# Manually creating a new row
# Assigning 'team' to the player and "" to the position.  This is where the aggregates will live
home_players.loc[len(home_players)+1,'Player'] = "Team"
home_players.loc[len(home_players)+1,'Position'] = ""

away_players.loc[len(away_players)+1,'Player'] = "Team"
away_players.loc[len(away_players)+1,'Position'] = ""